In [1]:
#%%
import sys; sys.path.append("../")
from spark_wrapper.spark_wrapper import SparkWrapper
from dotenv import load_dotenv; load_dotenv()
import os

ak = os.getenv("AWS_ACCESS_KEY")
sk = os.getenv("AWS_SECRET_KEY")
ep = os.getenv("AWS_ENDPOINT_URL")

pg_user = os.getenv("PG_USER")
pg_pwd = os.getenv("PG_PWD")
pg_host = os.getenv("PG_HOST")
pg_port = os.getenv("PG_PORT")
pg_db = os.getenv("PG_DB")

In [2]:
sw = SparkWrapper(num_cors=2, memory_gb=2) \
    .set_s3_conf(ak, sk, ep) \
    .set_pg_conf(pg_user, pg_pwd, pg_host, pg_port, pg_db) 

spark = sw.create_session()

:: loading settings :: url = jar:file:/home/luis/Documents/driva/utils/spark_wrapper/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/luis/.ivy2/cache
The jars for the packages stored in: /home/luis/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c4fb2eeb-de5d-482c-bf7f-930e4c0b4381;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 675ms :: artifacts dl 28ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            m

In [3]:
df = spark.read.parquet('s3a://drivalake/sites/silver/whois/brazil/')
df.persist()
df.show()

24/09/24 13:06:10 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
24/09/24 13:06:21 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


+--------------------+--------------------+--------------------+--------------------+------+-------------------------+--------------------------+--------------------------------------+------------------------+-------------------------------------+--------------------+------------------------+-------------------------------------+
|              domain|   registration_date|   last_changed_date|     expiration_date|status|entity_registrant_doc_num|entity_registrant_doc_type|entity_registrant_legal_representative|entity_technical_doc_num|entity_technical_legal_representative|         nameservers|entity_registrar_doc_num|entity_registrar_legal_representative|
+--------------------+--------------------+--------------------+--------------------+------+-------------------------+--------------------------+--------------------------------------+------------------------+-------------------------------------+--------------------+------------------------+-------------------------------------+
|   

In [6]:
df.rdd.getNumPartitions()

2

In [7]:
df.groupBy("expiration_date").count().show()

+--------------------+-----+
|     expiration_date|count|
+--------------------+-----+
|2025-09-18T18:34:55Z|    1|
|2025-09-09T02:34:10Z|    1|
|2024-11-30T02:12:21Z|    2|
|2025-02-15T12:00:27Z|    1|
|2025-06-04T18:08:28Z|    1|
|2025-04-20T16:19:05Z|    1|
|2025-02-12T20:14:08Z|    1|
|2025-06-11T12:37:19Z|    1|
|2025-03-15T11:21:42Z|    1|
|2033-07-27T16:51:47Z|    1|
|2026-04-03T18:39:26Z|    2|
|2026-06-14T21:23:56Z|    1|
|2033-03-25T13:28:32Z|    1|
|2024-12-10T20:28:57Z|    2|
|2025-08-21T18:02:47Z|    2|
|2025-08-29T18:38:52Z|    1|
|2025-01-09T17:25:48Z|    2|
|2025-11-20T17:44:31Z|    1|
|2025-04-16T18:46:24Z|    2|
|2026-03-13T19:41:56Z|    1|
+--------------------+-----+
only showing top 20 rows

